In [1]:
import numpy as np
import tensorflow as tf
from integrated import BasicRNNCell, MultiRNNCell, dynamic_rnn
#from core_rnn_cell_impl import BasicRNNCell, GRUCell, MultiRNNCell
#from rnn import dynamic_rnn
%autosave 0

Autosave disabled


In [2]:
i = np.zeros([10, 10, 10], dtype=np.float32)
num_layers = 3

g = tf.Graph()
with g.as_default():
    cell_base = BasicRNNCell(100)
    cell = MultiRNNCell([BasicRNNCell(100) for _ in range(num_layers)])
    output, state = dynamic_rnn(cell, i, dtype=tf.float32)
    print('\nBasicRNN:\nNumber of state outputs: {}\n{}'.format(len(state), state))


BasicRNN:
Number of state outputs: 3
(<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 100) dtype=float32>, <tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 100) dtype=float32>, <tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 100) dtype=float32>)


In [21]:
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.util import nest
from tensorflow.python.ops import math_ops

def linear2(args, output_size, bias, bias_start=0.0):
  """Linear map: sum_i(args[i] * W[i]), where W[i] is a variable.
  Args:
    args: a 2D Tensor or a list of 2D, batch x n, Tensors.
    output_size: int, second dimension of W[i].
    bias: boolean, whether to add a bias term or not.
    bias_start: starting value to initialize the bias; 0 by default.
  Returns:
    A 2D Tensor with shape [batch x output_size] equal to
    sum_i(args[i] * W[i]), where W[i]s are newly created matrices.
  Raises:
    ValueError: if some of the arguments has unspecified or wrong shape.
  """
  if args is None or (nest.is_sequence(args) and not args):
    raise ValueError("`args` must be specified")
  if not nest.is_sequence(args):
    args = [args]

  # Calculate the total size of arguments on dimension 1.
  total_arg_size = 0
  shapes = [a.get_shape() for a in args]
  for shape in shapes:
    if shape.ndims != 2:
      raise ValueError("linear is expecting 2D arguments: %s" % shapes)
    if shape[1].value is None:
      raise ValueError("linear expects shape[1] to be provided for shape %s, "
                       "but saw %s" % (shape, shape[1]))
    else:
      total_arg_size += shape[1].value

  dtype = [a.dtype for a in args][0]

  # Now the computation.
  weights = vs.get_variable(
      "weights", [total_arg_size, output_size], dtype=dtype)
  if len(args) == 1:
    res = math_ops.matmul(args[0], weights)
  else:
    res = math_ops.matmul(array_ops.concat(args, 1), weights)
  if not bias:
    return res
  biases = vs.get_variable(
      "biases", [output_size],
      dtype=dtype,
      initializer=init_ops.constant_initializer(bias_start, dtype=dtype))
  return nn_ops.bias_add(res, biases)

In [41]:
def linear3(args, output_size, bias, bias_start=0.0):
  """Linear map: sum_i(args[i] * W[i]), where W[i] is a variable.
  Args:
    args: a 2D Tensor or a list of 2D, batch x n, Tensors.
    output_size: int, second dimension of W[i].
    bias: boolean, whether to add a bias term or not.
    bias_start: starting value to initialize the bias; 0 by default.
  Returns:
    A 2D Tensor with shape [batch x output_size] equal to
    sum_i(args[i] * W[i]), where W[i]s are newly created matrices.
  Raises:
    ValueError: if some of the arguments has unspecified or wrong shape.
  """
  if args is None or (nest.is_sequence(args) and not args):
    raise ValueError("`args` must be specified")
  if not nest.is_sequence(args):
    args = [args]

  # Calculate the total size of arguments on dimension 1.
  total_arg_size = 0
  shapes = [a.get_shape() for a in args]
  for shape in shapes:
    if shape.ndims != 2:
      raise ValueError("linear is expecting 2D arguments: %s" % shapes)
    if shape[1].value is None:
      raise ValueError("linear expects shape[1] to be provided for shape %s, "
                       "but saw %s" % (shape, shape[1]))
    else:
      total_arg_size += shape[1].value

  dtype = [a.dtype for a in args][0]

  # Now the computation.
  scope = vs.get_variable_scope()
  with vs.variable_scope(scope) as outer_scope:
    weights = vs.get_variable(
        "weights", [total_arg_size, output_size], dtype=dtype)
    if len(args) == 1:
      res = math_ops.matmul(args[0], weights)
    else:
      res = math_ops.matmul(array_ops.concat(args, 1), weights)
    if not bias:
      return res
    with vs.variable_scope(outer_scope) as inner_scope:
      inner_scope.set_partitioner(None)
      biases = vs.get_variable(
          "biases", [output_size],
          dtype=dtype,
          initializer=init_ops.constant_initializer(bias_start, dtype=dtype))
    return nn_ops.bias_add(res, biases)


In [51]:
tf.reset_default_graph()
scope = vs.get_variable_scope()
num_units = 10
inputs = tf.zeros((10,10),dtype=tf.float32)
scope = vs.get_variable_scope()
with vs.variable_scope(scope) as outer_scope:
  weights = tf.get_variable("weights",[10,10],dtype=tf.float32)
# biases = tf.Variable(tf.zeros((10,10)),name="biases")
#  output = new_state = act(W * input + U * state + B).
#output = linear2([inputs, state], num_units, True))
scope.reuse_variables()
output = linear3(inputs, num_units, None)

In [43]:
output

<tf.Tensor 'MatMul:0' shape=(10, 100) dtype=float32>

In [55]:
with tf.Session() as sess:
    tf.global_variables_initializer()
    weights = vs.get_variable(
      "weights", [10, 10], dtype=tf.float32)
    out = sess.run(output,feed_dict = {
        weights: np.zeros((10,10)) 
    })

In [56]:
out

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)